In [10]:
import cv2
import os
import numpy as np
import face_recognition
import pandas as pd
from datetime import datetime

In [11]:
path = "images"

In [12]:
mylist = os.listdir(path)
mylist

['Bhavana.jpg', 'Smriti.png']

In [13]:
imgs = []
classnames = []

for img in mylist:
    # print(img)
    img_path = os.path.join(path,img)
    # print(img_path)
    curr_img = cv2.imread(img_path)
    # print(curr_img)
    imgs.append(curr_img)
    # print(imgs)
    # classnames.append(img[:-4]) 
    #  or
    classnames.append(img.split(".")[0])
print(len(imgs))
print(classnames)
print(imgs)

2
['Bhavana', 'Smriti']
[array([[[  2,  11,  14],
        [  2,  11,  14],
        [  3,  12,  15],
        ...,
        [  1,   0,   4],
        [  0,   0,   3],
        [  0,   0,   1]],

       [[  2,  11,  14],
        [  3,  12,  15],
        [  3,  12,  15],
        ...,
        [  1,   0,   4],
        [  0,   0,   3],
        [  0,   0,   1]],

       [[  3,  12,  15],
        [  3,  12,  15],
        [  3,  12,  15],
        ...,
        [  1,   0,   4],
        [  0,   0,   3],
        [  0,   0,   1]],

       ...,

       [[219, 223, 228],
        [224, 227, 231],
        [144, 140, 139],
        ...,
        [ 45,  43,  55],
        [  0,   0,   6],
        [  7,   7,  13]],

       [[207, 211, 216],
        [167, 170, 174],
        [194, 190, 189],
        ...,
        [ 54,  52,  64],
        [  0,   0,   6],
        [  6,   6,  12]],

       [[141, 144, 152],
        [ 57,  60,  64],
        [ 79,  75,  74],
        ...,
        [ 21,  20,  30],
        [ 30,  29,  38],

In [14]:
def face_encoding(images):
    encoded_list = []
    for img in imgs:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        face_in_frame = face_recognition.face_locations(img)
        # print(face_in_frame)
        encode = face_recognition.face_encodings(img,face_in_frame)[0]
        # print(encode)
        encoded_list.append(encode)
    return encoded_list


In [15]:
encoded_lis_known_faces = face_encoding(imgs)
print(len(encoded_lis_known_faces))

2


In [16]:
video = cv2.VideoCapture(0)
results = []
while True:
    suc,img = video.read()
    img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    face_in_frame = face_recognition.face_locations(img1)
    # print(face_in_frame)
    encoded_face = face_recognition.face_encodings(img,face_in_frame)
    # print(encoded_face)
    for loc,enc in zip(face_in_frame,encoded_face):
        matches = face_recognition.compare_faces(encoded_lis_known_faces,enc)
        # print(matches)
        face_dist = face_recognition.face_distance(encoded_lis_known_faces,enc)
        # print(face_dist)
        matchindex = np.argmin(face_dist)
        # print(matchindex)
        if matches[matchindex]:
            name = classnames[matchindex]
            y1, x2, y2, x1 = loc
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(img, name, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2)

            now = datetime.now()
            date_str = now.strftime("%Y-%m-%d")
            time_str = now.strftime("%H:%M:%S")

            if not any(r["Name"] == name and r["Date"] == date_str for r in results):
                results.append({"Name": name, "Date": date_str, "Time": time_str})
        else:
            y1, x2, y2, x1 = loc
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(img, "unknown", (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2)

    cv2.imshow("WEBCAM",img)
    if cv2.waitKey(1) & 0XFF == ord("q"):
        break
video.release()
cv2.destroyAllWindows()

In [17]:
print(results)

[{'Name': 'Bhavana', 'Date': '2026-02-23', 'Time': '14:22:30'}, {'Name': 'Smriti', 'Date': '2026-02-23', 'Time': '14:22:32'}]


In [18]:

# Load existing file if it exists, otherwise create a new DataFrame
if os.path.exists("attendance.xlsx"):
    df_existing = pd.read_excel("attendance.xlsx")
else:
    df_existing = pd.DataFrame(columns=["Name", "Date", "Time"])

# Append only new records (preserve old data)
for r in results:
    already_marked = ((df_existing["Name"] == r["Name"]) & (df_existing["Date"] == r["Date"])).any()
    if not already_marked:
        df_existing = pd.concat([df_existing, pd.DataFrame([r])], ignore_index=True)

# Save back to Excel
df_existing.to_excel("attendance.xlsx", index=False)
print("Attendance written to attendance.xlsx")


Attendance written to attendance.xlsx
